In [ ]:
import json 
import pandas as pd
import re
import itertools as it
from google.colab import drive
from bs4 import BeautifulSoup
import requests
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import of Immobiliare json file
with open('/content/drive/MyDrive/Progetto Data Management/Data/immobiliare.json', 'r') as file:
    immobiliare = json.load(file)

# Import of Subito json file
with open('/content/drive/MyDrive/Progetto Data Management/Data/subito.json', 'r') as file:
    subito = json.load(file)

In [ ]:
# the keys are str type, make it int
subito = {int(indice) : valore for indice, valore in subito.items()}
immobiliare = {int(indice) + len(subito): valore for indice, valore in immobiliare.items()}

In [ ]:
# we fix the types from float to int
import numpy as np
df_subito = pd.DataFrame(subito).transpose()
df_subito = df_subito.replace({np.nan: None})
for i in range(len(df_subito)):
  if df_subito.loc[i, 'Price/month'] is not None:
    df_subito.loc[i, 'Price/month'] = int(df_subito.loc[i, 'Price/month'])
  if df_subito.loc[i, 'Locals'] is not None: 
    df_subito.loc[i, 'Locals'] = int(df_subito.loc[i, 'Locals'])
  if df_subito.loc[i, 'Bathrooms'] is not None:
    df_subito.loc[i, 'Bathrooms'] = int(df_subito.loc[i, 'Bathrooms'])
  if df_subito.loc[i, 'Floor'] is not None:
    df_subito.loc[i, 'Floor'] = int(df_subito.loc[i, 'Floor'])
# then we update the dictionary
subito = df_subito.to_dict(orient="index")

# we fix the types from float to int

df_immobiliare = pd.DataFrame(immobiliare).transpose()
df_immobiliare = df_immobiliare.replace({np.nan: None})
for i in range(len(subito), len(subito)+len(immobiliare)):
  if df_immobiliare.loc[i, 'Price/month'] is not None:
    df_immobiliare.loc[i, 'Price/month'] = int(df_immobiliare.loc[i, 'Price/month'])   
  if df_immobiliare.loc[i, 'Size (m^2)'] is not None:
    df_immobiliare.loc[i, 'Size (m^2)'] = int(df_immobiliare.loc[i, 'Size (m^2)']) 
  if df_immobiliare.loc[i, 'Bedrooms number'] is not None:
    df_immobiliare.loc[i, 'Bedrooms number'] = int(df_immobiliare.loc[i, 'Bedrooms number']) 

# then we update the dictionary
immobiliare = df_immobiliare.to_dict(orient="index")    

In [ ]:
# Creation of a dataframe with common keys

df_subito = pd.DataFrame(subito).transpose()
df_immobiliare = pd.DataFrame(immobiliare).transpose()
common_keys = ['Price/month', 'Size (m^2)', 'Locals', 'Bathrooms', 'Floor', 'Description', 'If_agency', 'URL', 'District']
df_immobiliare = df_immobiliare[common_keys]
df_subito = df_subito[common_keys]

df_integration = pd.concat([df_subito, df_immobiliare]).reset_index()
len(df_integration) # number of ads in the integrated dataset

2732

### Check for duplicates in the integrated dataset

In [ ]:
dup = df_integration[df_integration.duplicated(subset = ['Price/month', 'Size (m^2)', 'Locals', 'Bathrooms', 'Floor', 'If_agency', 'District'], keep=False)]

dup = dup[dup["Description"].notnull()]

groups = dup.groupby(by = ['Price/month', 'Size (m^2)', 'Locals', 'Bathrooms', 'Floor', 'If_agency', 'District'], sort = False).groups


desc = dup["Description"].astype(str)

for i in desc.index: 
  desc[i] = re.sub(r'[^\w]+', ' ', desc[i])

desc = desc.str.lower().str.split() 

for i in desc.index: 
    desc[i] = set(desc[i])

lista = [] 
for val in groups.values():
  lista.append(desc.loc[val])

final_dup = [] 

for i in range(0, len(lista)):
  for x, y in it.combinations(lista[i], 2):
    if len(x & y)/len(x.union(y)) >= 0.9:
      final_dup.append(lista[i][lista[i] == x].index[0])
      final_dup.append(lista[i][lista[i] == y].index[0])

new = df_integration.iloc[final_dup].drop_duplicates() 

df_integration2 = df_integration.drop(index=new.index).reset_index(drop=True)
len(df_integration2) # we "lost" 35 duplicates between subito and immobiliare

2697

In [ ]:
ads_index = df_integration2['index'].tolist()
dummy = {chiave: subito.get(chiave, immobiliare.get(chiave)) for chiave in ads_index}
integration = {}
# reset index, some keys are not unique
count = 0
for i in dummy.keys():
  integration[count] = dummy[i]
  count += 1

## **Enrichment with Google News**
### Let's select the address and extrapolate title and url of relative news articles in the last year from the MilanoToday website.

In [ ]:
vie_modificate = []

for i in integration.keys():
  vie_modificate.append(integration[i]["Address"].replace(" ", "+"))

article_url = []
article_title = []

headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.5672.134 Safari/537.36'})

for street in vie_modificate:
  if street != "":
    link = 'https://news.google.com/search?q="' + str(street) + '"+site%3Amilanotoday.it%20when%3A1y&hl=it&gl=IT&ceid=IT%3Ait'
    r = requests.get(link, headers=headers)
    page_html = BeautifulSoup(r.text, 'html.parser')
    house_containers = page_html.find_all('div', class_='NiLAwe y6IFtc R7GTQ keNKEd j7vNaf nID9nc')

    if house_containers != []:
      lista_url_temp = []
      lista_article_temp = []

      for article in house_containers:

        link = "https://news.google.com" + article.find('a')['href'][1:]
        lista_url_temp.append(link)
      
        title = article.find('h3').text
        lista_article_temp.append(title)

      article_title.append(lista_article_temp)
      article_url.append(lista_url_temp)
    else:
      article_title.append("")
      article_url.append("")      
  else:
    article_title.append("")
    article_url.append("")

### Index to check the validity of the news-articles-column by address

In [ ]:
# 54.6 % of the listings have news articles related to their address.

count = 0
for i in article_title:
  if i != "":
    count += 1
rap1 = count / len(article_title)
print(rap1)

0.5461624026696329


In [ ]:
# 14.2 % of the listings don't specify the address of the house in the description/title.

cou = 0
for i in integration.keys():
  if integration[i]['Address'] == "":
    cou += 1
rap2 = cou / len(article_title)
print(rap2)

0.14200964034111976


In [ ]:
# So 31.2 % of the listings don't have news-articles related to their address.

print(1 - rap1 - rap2)

0.31182795698924737


### **Final data enrichment**

In [ ]:
for i in integration.keys():
  integration[i]["News"] = []

  if type(article_title[i]) == list:
    for m in range(len(article_title[i])):
      integration[i]["News"].append({"title" : article_title[i][m], "URL" : article_url[i][m]})

  else:
    integration[i]["News"].append({"title" : article_title[i], "URL" : article_url[i]})


In [ ]:
integration = list(integration.values())

with open("/content/drive/MyDrive/Progetto Data Management/Data/integratedDB.json", "w") as file:
  json.dump(integration, file)